In [160]:
import requests
from bs4 import BeautifulSoup
import pymysql
from selenium import webdriver
import time

In [161]:
def keyword_search(arr):
    a = arr.split()
    answer_cnt = 0
    for i in a:
        count = 0;
        for j in a:
            j = j.replace('"', '')
            if i in j and len(i) > 1:
                count+=1;
                if answer_cnt < count:
                    answer_cnt = count
                    answers = i
    return answers

In [162]:
def head_lines(link):
        title = link.text
        link = 'https://news.naver.com'+link['href']            
        ord_id = link.split('&')[3] + '&' + link.split('&')[4]
        url = link
        
        browser.get(url)
        
        soup2 = BeautifulSoup(browser.page_source, 'lxml')
        
        categori = soup2.find('em' , {'class' : 'guide_categorization_item'})

        if categori and want_thing == categori.get_text():
            categoris = categori.get_text()
        elif '모두' in want_thing:
            categoris = categori.get_text()
        else:
            categoris = -1
        title = title.replace('"', "'")
        
        return ord_id, title, press, categoris, link

In [163]:
def detail_table(link):
    soup = BeautifulSoup(browser.page_source, 'lxml')
    prev_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        curr_height = browser.execute_script("return document.body.scrollHeight")    
        if curr_height == prev_height:    
            break    
        prev_height = curr_height

    reporter = soup.find('div', {'class' : 'journalistcard_summary_name'})
    if reporter:
        reporter = reporter.text
    else:
        reporter = 'None'
    date = soup.find('span', {'class' : 't11'}).get_text()
    comment_cnt = 0
    try:
        comment_cnt = browser.find_element_by_css_selector('span.u_cbox_info_txt').text # 전체 댓글
    except:
        comment_cnt = browser.find_element_by_css_selector('em.simplecmt_num').text
        pass
    comment = int(comment_cnt.replace(',', ''))
    #comment_cnt = browser.find_element_by_css_selector('span.u_cbox_info_txt').text # 현재 댓글
    
    texts = soup.find('div', {'class' : '_article_body_contents'}).get_text()
    keyword = keyword_search(texts)
    
    return(reporter, date, comment, keyword)

In [164]:
# 웹 스크래핑 부분

what_date = input('어느 일자의 기사들을 보시겠습니까 ("0720" 과 같이 입력하여 주십시오.)')

db = pymysql.connect(host='localhost', port=3306, user='root', passwd='goodo011@', db='naver_news', charset='utf8')
cursor = db.cursor()
text = []

cursor.execute('drop table if exists headlines');
cursor.execute('drop table if exists detail_news');

sql = """
create table headlines
	(
	ord_id varchar(30) not null primary key,
    title varchar(300) not null,
    press varchar(100) not null,
    categori varchar(40) not null,
    url varchar(500) not null
    );
"""

cursor.execute(sql)

sql = """
create table detail_news
	(
    ord_id varchar(30) not null,
    reporter varchar(50) not null,
    date varchar(100),
    keyword varchar(20),
    comment_cnt int
    );
"""
cursor.execute(sql)

어느 일자의 기사들을 보시겠습니까 ("0720" 과 같이 입력하여 주십시오.)0520


0

In [165]:
browser = webdriver.Chrome()
url = f'https://news.naver.com/main/ranking/popularDay.naver?date=2021{what_date}'

browser.get(url)

soup = BeautifulSoup(browser.page_source, 'lxml')

rankingnews = soup.find_all('div', {'class' : 'rankingnews_box'})

In [166]:
try:
    want_thing = input('정치, 경제, 사회, 생활, 세계, IT 중에 보고싶은 카테고리를 입력하세요.(모든 카테고리를 희망한다면 "모두"를 입력하세요) : ')
    what_press = input('어느 언론사의 기사들을 보시겠습니까. (모든 언론사를 원하신다면 "모두" 라고 입력하십시오)')
    cnt = int(input('각 언론사에서 몇개의 기사들을 보시겠습니까. (1~5의 값을 입력해주세요)'))


    for ranking in rankingnews:
        press = ranking.find('strong', {'class' : 'rankingnews_name'}).text
        ranks = ranking.find_all('li')
        count = 0
        for rank in ranks:
            if what_press not in press and what_press != '모두':
                continue

            count+=1
            if count > cnt:
                break;
            link = rank.find('a' , {'class' : "list_title nclicks('RBP.rnknws')"})
            
            if not link:
                continue
            elif link:
                ord_id, title, press, categoris, link = head_lines(link)
                if categoris == -1:
                    continue
                print(ord_id, title, press, categoris, link)
                sql = f"""insert into headlines values("{ord_id}", "{title}", "{press}", '{categoris}', '{link}');"""
                cursor.execute(sql)
                
                reporter, date, comment_cnt, keyword = detail_table(link)
                print(title, reporter, date, comment_cnt, keyword)
                print('---------------------------------------------------------------------------------------------\n\n')
                sql = f"""insert into detail_news values("{ord_id}", "{reporter}", "{date}", '{keyword}', {comment_cnt});"""
                cursor.execute(sql)
                time.sleep(1)
finally:
    db.commit()
    db.close()

정치, 경제, 사회, 생활, 세계, IT 중에 보고싶은 카테고리를 입력하세요.(모든 카테고리를 희망한다면 "모두"를 입력하세요) : 모두
어느 언론사의 기사들을 보시겠습니까. (모든 언론사를 원하신다면 "모두" 라고 입력하십시오)모두
각 언론사에서 몇개의 기사들을 보시겠습니까. (1~5의 값을 입력해주세요)1
oid=277&aid=0004905664 김태희·하정우·소유 등 연예인들 잇따라 보유건물 처분… 왜? 아시아경제 경제 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=277&aid=0004905664&rankingType=RANKING
김태희·하정우·소유 등 연예인들 잇따라 보유건물 처분… 왜? 류태민 2021.05.20. 오전 10:56 343 빌딩
---------------------------------------------------------------------------------------------


oid=001&aid=0012403700 대한민국 최고의 전통미인 '미스춘향 진'에 김민설씨 연합뉴스 사회 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=001&aid=0012403700&rankingType=RANKING
대한민국 최고의 전통미인 '미스춘향 진'에 김민설씨 백도인 2021.05.20. 오전 1:09 1337 남원시
---------------------------------------------------------------------------------------------


oid=437&aid=0000266676 71세 할머니와 결혼한 18살…'6년간 매일 사랑에 더 빠져' JTBC 세계 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=437&a

월급6700만원 성형외과 의사가 '형편어렵다'며 2억8400만원 세금 체납…서울시, '금융채무불이행자' 등록한다 류인하 주간경향 기자 2021.05.20. 오전 11:16 446 서울
---------------------------------------------------------------------------------------------


oid=003&aid=0010507882 '美기업들 '이재용 석방해달라' 文대통령에 서한 전달' FT 뉴시스 세계 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=003&aid=0010507882&rankingType=RANKING
'美기업들 '이재용 석방해달라' 文대통령에 서한 전달' FT 이혜원2 기자 2021.05.20. 오후 3:40 780 대통령
---------------------------------------------------------------------------------------------


oid=374&aid=0000244914 김진표 “부동산 공급 및 규제안 하나의 정책으로 조율후 발표” SBS Biz 정치 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=374&aid=0000244914&rankingType=RANKING
김진표 “부동산 공급 및 규제안 하나의 정책으로 조율후 발표” 김종윤 기자 2021.05.20. 오후 4:51 65 정책
---------------------------------------------------------------------------------------------


oid=422&aid=0000485924 '집단폭행 살해' 태권도 유단자들 징역형 확정 연합뉴스TV 사회 https://news.naver.com/main/ranking/read.naver?

oid=417&aid=0000695428 차량 뒤로 다가와 일부러 차 건드린 아이들… '경적 누르자 도망가' 머니S 생활 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=417&aid=0000695428&rankingType=RANKING
차량 뒤로 다가와 일부러 차 건드린 아이들… '경적 누르자 도망가' 빈재욱 기자 2021.05.20. 오후 12:49 134 들은
---------------------------------------------------------------------------------------------


oid=082&aid=0001092125 ‘2조 넘는 폭리’ 누워서 떡 먹는 부산 민자도로 부산일보 사회 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=082&aid=0001092125&rankingType=RANKING
‘2조 넘는 폭리’ 누워서 떡 먹는 부산 민자도로 김준용,손혜림 2021.05.19. 오후 7:33 133 운영
---------------------------------------------------------------------------------------------


oid=088&aid=0000703838 [속보] 대전 70대女 이어 속초 80대男도 화이자 백신 2차 접종 후 사망 '기저질환 X' 매일신문 사회 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=088&aid=0000703838&rankingType=RANKING
[속보] 대전 70대女 이어 속초 80대男도 화이자 백신 2차 접종 후 사망 '기저질환 X' 황희진 2021.05.20. 오후 5:09 167 화이자
------------------------------

'세계 최초 배달앱' 배달통, 역사 속으로 이현석 2021.05.20. 오전 10:42 5 배달
---------------------------------------------------------------------------------------------


oid=353&aid=0000039559 자안바이오·형지I&C, BW 사모 대신 공모로 정면돌파 중앙SUNDAY 경제 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=353&aid=0000039559&rankingType=RANKING
자안바이오·형지I&C, BW 사모 대신 공모로 정면돌파 None 2021.05.15. 오전 12:22 2 투자
---------------------------------------------------------------------------------------------


oid=145&aid=0000015410 백신 접종 다시 속도···화이자 백신 43만8000회분 도착 레이디경향 생활 https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=145&aid=0000015410&rankingType=RANKING
백신 접종 다시 속도···화이자 백신 43만8000회분 도착 김문석 기자 2021.05.19. 오후 4:13 10 백신
---------------------------------------------------------------------------------------------




In [159]:
SQL = "use naver_news"
df = pd.read_sql(SQL, db)
SQL = "SELECT * FROM headlines"
df = pd.read_sql(SQL, db)
df.to_csv('headlines.csv', sep=',', index=False, encoding='utf-8')
SQL = "SELECT * FROM detail_news"
df = pd.read_sql(SQL, db)
df.to_csv('detail_news.csv', sep=',', index=False, encoding='utf-8')
df

DatabaseError: Execution failed on sql: SELECT * FROM headlines
(0, '')
unable to rollback